In [1]:
import os
import requests

# Create the directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Download files using requests
url1 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet'
url2 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet'

# Define the file paths
file_path1 = '../data/green_tripdata_2024-01.parquet'
file_path2 = '../data/green_tripdata_2024-02.parquet'

# Download and save the first file
response = requests.get(url1)
with open(file_path1, 'wb') as f:
    f.write(response.content)

# Download and save the second file
response = requests.get(url2)
with open(file_path2, 'wb') as f:
    f.write(response.content)

print('Files downloaded successfully!')

Files downloaded successfully!


In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/arturotowers/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)


mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as arturotowers

Initialized MLflow to track repo "arturotowers/nyc-taxi-time-prediction"

Repository arturotowers/nyc-taxi-time-prediction initialized!

https://dagshub.com/arturotowers/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/e458b17cf36e4062a9a98d2cb0799db8', creation_time=1726632310641, experiment_id='0', last_update_time=1726632310641, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [10]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("arturotowers/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)
# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [11]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [13]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [17]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/09/20 20:44:26 WARNING mlflow.xgboost: Failed to log dataset information to MLflow Tracking. Reason: Unable to allocate 1.03 GiB for an array with shape (276921689,) and data type float32



[0]	validation-rmse:7.79294                           
[1]	validation-rmse:6.89979                           
[2]	validation-rmse:6.31360                           
[3]	validation-rmse:5.93976                           
[4]	validation-rmse:5.70523                           
[5]	validation-rmse:5.56025                           
[6]	validation-rmse:5.47168                           
[7]	validation-rmse:5.41391                           
[8]	validation-rmse:5.37740                           
[9]	validation-rmse:5.35564                           
[10]	validation-rmse:5.33943                          
[11]	validation-rmse:5.32912                          
[12]	validation-rmse:5.32014                          
[13]	validation-rmse:5.31363                          
[14]	validation-rmse:5.30840                          
[15]	validation-rmse:5.30520                          
[16]	validation-rmse:5.29938                          
[17]	validation-rmse:5.29639                          
[18]	valid

2024/09/20 20:44:58 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 20:44:58 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\ctorr\OneDrive\Documentos\Arturo\Arturo HW ITESO\5to semestre\Proyecto Ciencia de datos\apis\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:44:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 20:45:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\ctorr\AppData\Local\Temp\tmp735z4od5\model, flavor: xgboost). Fall back to return ['xgboost==2.1.1']. Set lo

[0]	validation-rmse:8.75345                                                    
[1]	validation-rmse:8.42531                                                    
[2]	validation-rmse:8.12540                                                    
[3]	validation-rmse:7.85188                                                    
[4]	validation-rmse:7.60285                                                    
[5]	validation-rmse:7.37617                                                    
[6]	validation-rmse:7.17042                                                    
[7]	validation-rmse:6.98286                                                    
[8]	validation-rmse:6.81357                                                    
[9]	validation-rmse:6.66129                                                    
[10]	validation-rmse:6.52353                                                   
[11]	validation-rmse:6.39965                                                   
[12]	validation-rmse:6.28759            

...

2024/09/20 20:51:00 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 20:51:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\ctorr\OneDrive\Documentos\Arturo\Arturo HW ITESO\5to semestre\Proyecto Ciencia de datos\apis\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:51:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 20:51:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\ctorr\AppData\Local\Temp\tmpvau0otku\model, flavor: xgboost). Fall back to return ['xgboost==2.1.1']. Set lo

100%|██████████| 10/10 [06:53<00:00, 41.33s/trial, best loss: 5.164490570080507]


2024/09/20 20:51:19 WARNING mlflow.xgboost: Failed to log dataset information to MLflow Tracking. Reason: Unable to allocate 1.03 GiB for an array with shape (276921689,) and data type float32


[0]	validation-rmse:6.27731
[1]	validation-rmse:5.58283
[2]	validation-rmse:5.42148
[3]	validation-rmse:5.37310
[4]	validation-rmse:5.35609
[5]	validation-rmse:5.34691
[6]	validation-rmse:5.34334
[7]	validation-rmse:5.33833
[8]	validation-rmse:5.33386
[9]	validation-rmse:5.33146
[10]	validation-rmse:5.32544
[11]	validation-rmse:5.32141
[12]	validation-rmse:5.31531
[13]	validation-rmse:5.31222
[14]	validation-rmse:5.30835
[15]	validation-rmse:5.30107
[16]	validation-rmse:5.29424
[17]	validation-rmse:5.29125
[18]	validation-rmse:5.28379
[19]	validation-rmse:5.28205
[20]	validation-rmse:5.28253
[21]	validation-rmse:5.28152
[22]	validation-rmse:5.27303
[23]	validation-rmse:5.27197
[24]	validation-rmse:5.26793
[25]	validation-rmse:5.26337
[26]	validation-rmse:5.25905
[27]	validation-rmse:5.25465
[28]	validation-rmse:5.24800
[29]	validation-rmse:5.24596
[30]	validation-rmse:5.24495
[31]	validation-rmse:5.24338
[32]	validation-rmse:5.24239
[33]	validation-rmse:5.24176
[34]	validation-rmse:5.2

2024/09/20 20:51:26 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/20 20:51:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\ctorr\OneDrive\Documentos\Arturo\Arturo HW ITESO\5to semestre\Proyecto Ciencia de datos\apis\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:51:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/20 20:51:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\ctorr\AppData\Local\Temp\tmprfulk9sb\model, flavor: xgboost). Fall back to return ['xgboost==2.1.1']. Set logg

In [18]:
run_id = "43fb6f3393fc4a28b53ce039e0f19222"
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2024/09/20 20:55:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 5
Created version '5' of model 'nyc-taxi-model'.


In [19]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['challenger', 'champion'], creation_timestamp=1726882813057, current_stage='Staging', description='The model version 1 was transitioned to champion on 2024-09-20 20:55:16.707830', last_updated_timestamp=1726887317446, name='nyc-taxi-model', run_id='153069cf7f9840d996517f0976673cd5', run_link='', source='mlflow-artifacts:/28559c146958489da93a281d752509c1/153069cf7f9840d996517f0976673cd5/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [23]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"  # Asegúrate de que este alias es correcto

# Cargar el modelo usando el URI con el nombre y el alias
model_uri = f"models:/{model_name}@{alias}"

try:
    champion_version = mlflow.pyfunc.load_model(model_uri=model_uri)
    predictions = champion_version.predict(X_val)  # Asegúrate de que X_val está definido
    print(predictions)
except Exception as e:
    print(f"Error al cargar el modelo: {e}")

Error al cargar el modelo: The following failures occurred while downloading one or more artifacts from https://dagshub.com/arturotowers/nyc-taxi-time-prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/28559c146958489da93a281d752509c1/153069cf7f9840d996517f0976673cd5/artifacts/model:
##### File  #####
API request to https://dagshub.com/arturotowers/nyc-taxi-time-prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/28559c146958489da93a281d752509c1/153069cf7f9840d996517f0976673cd5/artifacts/model/ failed with exception HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /arturotowers/nyc-taxi-time-prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/28559c146958489da93a281d752509c1/153069cf7f9840d996517f0976673cd5/artifacts/model/ (Caused by ResponseError('too many 500 error responses'))


In [22]:
model = client.get_registered_model("nyc-taxi-model")
print(model)

<RegisteredModel: aliases={'challenger': '2', 'champion': '1'}, creation_timestamp=1726882812649, description='Model registry for the NYC Taxi Time Prediction Project', last_updated_timestamp=1726887317078, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1726887303265, current_stage='None', description='', last_updated_timestamp=1726887303265, name='nyc-taxi-model', run_id='43fb6f3393fc4a28b53ce039e0f19222', run_link='', source='mlflow-artifacts:/e458b17cf36e4062a9a98d2cb0799db8/43fb6f3393fc4a28b53ce039e0f19222/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>,
 <ModelVersion: aliases=[], creation_timestamp=1726882813057, current_stage='Staging', description='The model version 1 was transitioned to champion on 2024-09-20 20:55:16.707830', last_updated_timestamp=1726887317446, name='nyc-taxi-model', run_id='153069cf7f9840d996517f0976673cd5', run_link='', source='mlflow-artifacts:/28559c146958489da93a281d752509c1/153069cf7f9840d996517f0